In [102]:
import requests
import struct
import hashlib
import urllib3
import ipaddress
from Crypto.Cipher import ARC4

In [2]:
urllib3.disable_warnings()

In [3]:
proxies = {
    "http":"http://127.0.0.1:8080",
    "https":"http://127.0.0.1:8080"
}

# Setting
* `c2_gate` is the gate.php of the server
* `c2_key` is the encryption key extracted from the bot config file
* `botnet_id` is the botnet ID from the config file
* `header_size`, `md5_size`, `item_header_size` you shouldn't need to change


In [94]:
c2_gate = "http://192.168.86.86:8081/cp108/gate.php"
c2_key = "12345678"
header_size = 48
md5_size = 32
item_header_size = 16

# Defines from gen.php

In [184]:
BLT_COOKIES = 6
BLT_FILE = 5
BLT_GRABBED_HTTP = 3
BLT_HTTPS_REQUEST = 2
BLT_HTTP_REQUEST = 1
BLT_KEYLOGER = 7
BLT_PASSWORD = 8
BLT_SCREENSHOT = 9
BLT_UNKNOWN = 0
BOTNET_MAX_CHARS = 20
COMP_ID_MAX_CHARS = 100
CSR_BACKCONNECT_CRYPT_KEY = 0x55
CSR_BOT32_FILE = 1006
CSR_BOT64_FILE = 1001
CSR_BOT_FILE = 1000
CSR_BOT_VERSION = 0x1000c00
CSR_CP_VERSION = 0x1000800
CSR_ERR_BOT_VERSION = 3221225481.0
CSR_ERR_BUILD_INJECTS_FAILED = 3221225476.0
CSR_ERR_CANT_CONNECT_DATABASE = 3221225473.0
CSR_ERR_CONNECT_FAILED = 3221225477.0
CSR_ERR_COPY_FILE_FAILED = 3221225480.0
CSR_ERR_CREATE_PROCESS_FAILED = 3221225473.0
CSR_ERR_DUPLICATE_CONTROL_PANEL = 3221225478.0
CSR_ERR_FAILED = 3221225472.0
CSR_ERR_FILE_NOT_FOUND = 3221225479.0
CSR_ERR_INVALID_DATA = 3221225481.0
CSR_ERR_INVALID_FILE = 3221225475.0
CSR_ERR_INVALID_LICENSE_KEY = 3221225476.0
CSR_ERR_LICENSE_DISABLED = 3221225480.0
CSR_ERR_LICENSE_KEY_EXPIRED = 3221225477.0
CSR_ERR_LICENSE_KEY_NOT_SPECIFIED = 3221225474.0
CSR_ERR_PEPATH = 3221225482.0
CSR_ERR_PROCESS_TIMEOUT = 3221225483.0
CSR_ERR_SAVE_FILE_FAILED = 3221225474.0
CSR_ERR_SERVER_IP_NOT_SPECIFIED = 3221225475.0
CSR_ERR_START_UPDATE_TIMEOUT = 3221225478.0
CSR_ERR_STOPED_BOT_TIMEOUT = 3221225479.0
CSR_ERR_SUCCESS = 0x0
CSR_HVNC32_FILE = 1007
CSR_HVNC64_FILE = 1008
CSR_HVNC_VERSION = 0x1000c00
CSR_LAST_GEN = 1579182797
CSR_LIBSSL_FILE = 1002
CSR_LIBSSL_VERSION = 0x2
CSR_MAGIC_ANSWER = 50403020
CSR_NSS_FILE = 1005
CSR_NSS_VERSION = 0x1
CSR_POST_MAX_SIZE = 10
CSR_SQLITE_FILE = 1003
CSR_SQLITE_VERSION = 0x1
CSR_ZLIB_FILE = 1004
CSR_ZLIB_VERSION = 0x1
GATE_MAX_CHARS = 64
LOG_ID_BOT_DETECTED = 6
LOG_ID_CHECK_POST_MAX_SIZE = 5
LOG_ID_COLLECTOR = 10
LOG_ID_INSTALL_NSS_CERT = 4
LOG_ID_LOADER_UPDATE = 1
LOG_ID_PELOADER = 7
LOG_ID_PROCESS_INJECT = 8
LOG_ID_STEALER = 9
LOG_ID_WEBINJECTS_UPDATE = 3
MARKER_MAX_CHARS = 20
MAX_BOTNET_TIMEOUT = 60
MAX_NUM_GATES = 10
MIN_BOTNET_TIMEOUT = 1
PROCESS_INTEGRITY_HIGH = 3
PROCESS_INTEGRITY_LOW = 1
PROCESS_INTEGRITY_MEDIUM = 2
PROCESS_INTEGRITY_UNKNOWN = 0
RC4_MAX_KEY = 32
RC4_MIN_KEY = 8
SBCID_ADV_SERVERS = 11010
SBCID_BC_CLIENTPORT = 11041
SBCID_BC_HVNC_CLIENTPORT = 11042
SBCID_BC_IP = 11040
SBCID_BOTLOG = 200000
SBCID_BOTLOG_TYPE = 300000
SBCID_BOTNET = 10002
SBCID_BOT_ID = 10001
SBCID_BOT_VERSION = 10003
SBCID_COUNT_SCRIPTS = 11004
SBCID_DEBUG = 10022
SBCID_GET_FILE = 11014
SBCID_GET_FILE_VER = 11015
SBCID_HTTP_PROXY = 11013
SBCID_INTEGRITY_LEVEL = 10023
SBCID_IPV4_ADDRESSES = 10016
SBCID_IPV6_ADDRESSES = 10017
SBCID_LANGUAGE_ID = 10013
SBCID_LOADER_UPDATE = 11020
SBCID_LOADER_UPDATE_SUCCESS = 11021
SBCID_LOG_ERR_CODE = 11032
SBCID_LOG_ID = 11030
SBCID_LOG_ID_EXT = 11031
SBCID_LOG_MSG = 11033
SBCID_MARKER = 10025
SBCID_MD5_BOT = 10026
SBCID_NET_LATENCY = 10005
SBCID_NUM_MONITORS = 10024
SBCID_NUM_REPORTS = 100000
SBCID_OS_INFO = 10012
SBCID_PING = 10006
SBCID_PROCESS_LIST = 10020
SBCID_PROCESS_NAME = 10014
SBCID_PROCESS_USER = 10015
SBCID_SCRIPTS = 11003
SBCID_SCRIPT_ID = 11000
SBCID_SCRIPT_RESULT = 11002
SBCID_SCRIPT_STATUS = 11001
SBCID_SOURCE = 400000
SBCID_TIMEZONE = 10027
SBCID_TIME_LOCALBIAS = 800000
SBCID_TIME_SYSTEM = 600000
SBCID_TIME_TICK = 700000
SBCID_TITLE = 500000
SBCID_WEBFILTERS = 11011
SBCID_WEBINJECTS = 11012
SBCID_WEBINJECTS_UPDATE = 11022
SBCID_WEBINJECTS_UPDATE_SUCCESS = 11023

In [67]:
# Initialize RC4
def rc4_init(key):
    _hash = [None] * 256
    _box = [None] * 256
    _keylen = len(key)
    
    for x in range(256):
        _hash[x] = ord(key[x % _keylen])
        _box[x] = x
    
    y = 0
    for x in range(256):
        y = (y + _box[x] + _hash[x]) % 256
        tmp = _box[x]
        _box[x] = _box[y]
        _box[y] = tmp
    
    return _box

# Encrypt RC4
def rc4(data, key):
    _len = len(data)
    z = y = 0
    for x in range(_len):
        z = (z + 1) % 256
        y = (y + key[z]) % 256
        tmp = key[z]
        key[z] = key[y]
        key[y] = tmp
        data[x] = data[x] ^ (key[((key[z] + key[y]) % 256)])
    return data

# Visual Decrypt
def v_decrypt(data):
    _len = len(data)
    for x in reversed(range(1, _len)):
        data[x] = data[x] ^ data[x-1]
    return data

# Visual Encrypt
def v_encrypt(data):
    _len = len(data)
    for x in range(_len):
        data[x] = data[x] ^ data[x-1]
    return data

In [115]:
# Complete encrypt sequence
def encrypt(data):
    key = rc4_init(c2_key)
    data = v_encrypt(list(data))
    data = rc4(data, key=key)
    return data

# Complete decrypt sequence
def decrypt(data, skipVisual=False):
    key = rc4_init(c2_key)
    data = rc4(list(data), key=key)
    if skipVisual:
        return data
    else:
        return v_decrypt(data)

# Convert output from decrypt/encrypt into bytes object
def bytes2str(data):
    return bytes(bytearray(data))

In [116]:
# Send complete request to C2 and get decrypted data
def send_request(data, skipVisual=False):
    r = requests.post(c2_gate, data=bytes2str(encrypt(data)), proxies=proxies, verify=False)
    return bytes2str(decrypt(r.content, skipVisual))

In [117]:
def parse_response(data):
    header = data[:header_size]
    _md5 = header[md5_size:].hex()
    body = data[header_size:]
    parsed_data = {}
    x = 0
    while x < len(body):
        header_offset = x+item_header_size
        k = struct.unpack('IIII', body[x:header_offset])
        parsed_data[k[0]] = body[header_offset:header_offset+k[2]]
        x += item_header_size+k[2]
    return parsed_data
        

In [71]:
def pack_data(data):
    body = []
    for record_id, content in data.items():
        record_header = struct.pack('IIII', record_id, 0, len(content), 0)
        body.append(record_header + content)
    finished_body = b''.join(body)
    header = b''.join([b'0'*(md5_size), hashlib.md5(finished_body).digest()])
    return b''.join([header, finished_body])

In [72]:
def convert_to_int(hstr):
    return struct.unpack('I', bytearray.fromhex(hstr)) # 'C0A8641B'

In [120]:
def sr(data, skipVisual=False):
    resp = send_request(pack_data(data), skipVisual)
    if skipVisual:
        return resp
    return parse_response(resp)

# Talking to the bot
To talk to the bot you need to create a list of properties dictionary, for example `10002` is botnet id, while `10001` is bot's name.

Then pass the dictionary via `pack_data` to `send_request` like in example. 

Sometimes the C2 response will be parsable, but when its a binary, the parse_reponse will fail. Decoding binaries is not implemented here.

In [185]:
# New Bot
newbot_request = {
    SBCID_BOT_ID:b"BOTID",
    SBCID_BOTNET:b"BOTNET",
    SBCID_OS_INFO:struct.pack("BBHH", 2, 3, 9, 9),
    SBCID_BOT_VERSION:struct.pack("I", CSR_BOT_VERSION),
    SBCID_IPV4_ADDRESSES: int(ipaddress.IPv4Address('192.168.0.1')).to_bytes(4, byteorder='big'),
    SBCID_PROCESS_LIST:b"Item 1|Item2",
    SBCID_INTEGRITY_LEVEL:struct.pack("L", 1),
    SBCID_NUM_MONITORS:struct.pack("L", 1),
    SBCID_MARKER:b"BOTMARKER",
    SBCID_MD5_BOT: hashlib.md5(b"md5").digest(),
    SBCID_TIMEZONE:b"PST",
    SBCID_WEBINJECTS:struct.pack("L", True)
}

In [186]:
sr(newbot_request)

{11012: b'123',
 11004: b'\x00\x00\x00\x00',
 11010: b'http://192.168.86.86:8081/cp108/gate.php',
 11011: b'',
 11013: b'\x01\x00\x00\x00',
 1650751854: b'amcvdqtslrjbozrunjtnwozyhejltruhewshwhqxlklwscoasdodcw'}

In [194]:
# Script Report
script_report = {
    SBCID_BOT_ID:b"BOTID",
    SBCID_BOTNET:b"BOTNET",
    SBCID_SCRIPT_ID:bytes(bytearray.fromhex("9A1B74A332125248977AE171F7A51D23")),
    SBCID_SCRIPT_STATUS: struct.pack("L", 1),
    SBCID_SCRIPT_RESULT: b"Script result string"
}

In [195]:
sr(script_report)

{}

In [197]:
# File Request
file_request = {
    SBCID_BOT_ID:b"BOTID",
    SBCID_BOTNET:b"BOTNET",
    SBCID_GET_FILE: struct.pack('I', CSR_SQLITE_FILE),
    SBCID_GET_FILE_VER: struct.pack('I', 1)
}

In [198]:
sr(file_request, skipVisual=True)[:200]

b'\xeb\x03\x00\x00\x01\x00\x00\x00\x19\xfd#^{"\x0e\x00\x9c\xc1\x9b\x10\x00MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x0e\x1f\xba\x0e\x00\xb4\t\xcd!\xb8\x01L\xcd!This program cannot be run in DOS mode.\r\r\n$\x00\x00\x00\x00\x00\x00\x00PE\x00\x00L\x01\x12\x00\xd5)&]\x002\x0c\x00(\x10\x00\x00\xe0\x00\x06!\x0b\x01\x02\x19\x00v\t\x00\x00&\x0b\x00\x00\n\x00\x00\x00\x14\x00\x00\x00\x10\x00\x00\x00\x90\t'